# Matriz de Oferta y Utilizacion

El archivo a utilizar es descargable de la pagina web de DANE bajo Cuentas Nacionales Anuales

In [74]:
import requests
import pandas as pd

# Funcion para convertir los decimales de ingles a castellano
def decimal_converter(value):
    try:
        return float(value.replace(',', '.'))
    except ValueError:
        return value

# Definiendo el nombre de las columnas de la matriz
columns = [
    "Códigos CPC Vers. 2 A.C.", 
    "Concepto", 
    "Total oferta a precios comprador", 
    "Márgenes de comercio", 
    "Márgenes de transporte", 
    "Impuestos y derechos a las importaciones", 
    "IVA no deducible", 
    "Impuestos a los productos (excepto impuestos a importaciones e IVA no deducible)", 
    "Subvenciones a los productos", 
    "Oferta total precios básicos", 
    "Agricultura y actividades de servicios conexas ", 
    "Cultivo permanente de café", 
    "Ganadería, caza y actividades de servicios conexas ", 
    "Silvicultura y extracción de madera", 
    "Pesca y acuicultura", 
    "Extracción de carbón de piedra y lignito", 
    "Extracción de petróleo crudo y gas natural y actividades de apoyo para la extracción de petróleo y de gas natural", 
    "Extracción de minerales metalíferos", 
    "Extracción de otras minas y canteras", 
    "Actividades de apoyo para otras actividades de explotación de minas y canteras", 
    "Procesamiento y conservación de carne y productos cárnicos de bovinos, bufalinos, porcinos y otras carnes n.c.p.; procesamiento y conservación de carne y productos cárnicos de aves de corral y procesamiento y conservación de pescados, crustáceos y moluscos", 
    "Elaboración de aceites y grasas de origen vegetal y animal", 
    "Elaboración de productos lácteos", 
    "Elaboración de productos de molinería, almidones y productos derivados del almidón; elaboración de productos de panadería; elaboración de macarrones, fideos, alcuzcuz, y productos farináceos similares y elaboración de alimentos preparados para animales", 
    "Elaboración de productos de café", 
    "Elaboración de azúcar y elaboración de panela", 
    "Elaboración de cacao, chocolate y productos de confitería", 
    "Procesamiento y conservación de frutas, legumbres, hortalizas y tubérculos; elaboración de otros productos alimenticios (platos preparados y conservados mediante enlatado o congelado, elaboración de sopas y caldos en estado sólidos, polvo o instantáneas entre otros)", 
    "Elaboración de bebidas (incluido el hielo) y elaboración de productos de tabaco", 
    "Preparación, hilatura, tejeduría y acabado de productos textiles; fabricación de otros productos textiles, Confección de prendas de vestir", 
    "Curtido y recurtido de cueros; fabricación de calzado; fabricación de artículos de viaje, maletas, bolsos de mano y artículos similares, y fabricación de artículos de talabartería y guarnicionería; adobo y teñido de pieles", 
    "Transformación de la madera y fabricación de productos de madera y de corcho, excepto muebles; fabricación de artículos de cestería y espartería", 
    "Fabricación de papel, cartón y productos de papel y de cartón", 
    "Actividades de impresión; producción de copias a partir de grabaciones originales (Copia a partir de un original en CD, DVD, Bluray)", 
    "Coquización, fabricación de productos de la refinación del petróleo y actividades de mezcla de combustibles", 
    "Fabricación de sustancias químicas básicas, abonos y compuestos inorgánicos nitrogenados, plásticos y caucho sintético en formas primarias; fabricación de otros productos químicos; fabricación de fibras sintéticas y artificiales; fabricación de productos farmacéuticos, sustancias químicas medicinales y productos botánicos de uso farmacéutico", 
    "Fabricación de productos de caucho y de plástico", 
    "Fabricación de otros productos minerales no metálicos", 
    "Fabricación de productos metalúrgicos básicos; fabricación de productos elaborados de metal, excepto maquinaria y equipo", 
    "Fabricación de aparatos y equipo eléctrico; fabricación de productos informáticos, electrónicos y ópticos", 
    "Fabricación de maquinaria y equipo n.c.p.; instalación, mantenimiento y reparación especializado de maquinaria y equipo", 
    "Fabricación de vehículos automotores, remolques y semirremolques; fabricación de otros tipos de equipo de transporte", 
    " Fabricación de muebles, colchones y somieres", 
    "Otras industrias manufactureras", 
    "Generación de energía eléctrica; transmisión de energía eléctrica y distribución y comercialización de energía eléctrica", 
    "Producción de gas; distribución de combustibles gaseosos por tuberías; suministro de vapor y aire acondicionado", 
    "Captación, tratamiento y distribución de agua", 
    "Evacuación y tratamiento de aguas residuales; recolección, tratamiento y disposición de desechos y actividades de saneamiento ambiental y otros servicios de gestión de desechos", 
    "Recuperación de materiales (reciclaje)", 
    "Construcción de edificaciones residenciales y no residenciales", 
    "Construcción de carreteras y vías de ferrocarril, de proyectos de servicio público y de otras obras de ingeniería civil", 
    "Actividades especializadas para la construcción de edificaciones y obras de ingeniería civil (Alquiler de maquinaría y equipo de construcción con operadores)", 
    "Comercio al por mayor y en comisión o por contrata; comercio al por menor (incluso el comercio al por menor de combustibles); comercio de vehículos automotores y motocicletas, sus partes, piezas y accesorios", 
    "Mantenimiento y reparación de vehículos automotores y motocicletas", 
    "Transporte terrestre y transporte por tuberías", 
    "Transporte acuático", 
    "Transporte aéreo", 
    "Almacenamiento y actividades complementarias al transporte", 
    "Actividades de correo y de servicios de mensajería", 
    "Alojamiento y servicios de comida", 
    "Información y comunicaciones", 
    "Actividades financieras y de seguros", 
    "Actividades inmobiliarias", 
    "Actividades profesionales, científicas y técnicas", 
    "Actividades de servicios administrativos y de apoyo", 
    "Administración pública y defensa; planes de seguridad social de afiliación obligatoria", 
    "Educación de mercado", 
    "Educación de no mercado", 
    "Actividades de atención de la salud humana y de servicios sociales", 
    "Actividades artísticas, de entretenimiento y recreación y otras actividades de servicios", 
    "Actividades de los hogares individuales en calidad de empleadores", 
    "N/A 1",
    "Producción a precios básicos Total", 
    "Producción a precios básicos Para uso final propio", 
    "Producción a precios básicos Otra de no mercado", 
    "Producción a precios básicos De mercado", 
    "N/A 2",
    "Importaciones Ajustes  CIF/FOB sobre importaciones", 
    "Importaciones Bienes", 
    "Importaciones Servicios"
]


In [75]:
# Haciendo el request sin validar el certificado de seguridad
url = 'https://www.dane.gov.co/files/investigaciones/boletines/pib/cuentas-nal-anuales/oferta-utilizacion-precios-constantes-2020provisional.xlsx'
r = requests.get(url, verify=False)
open('temp.xls', 'wb').write(r.content)

C:\Users\patricia.ayuso\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.dane.gov.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


3191011

## Leyendo el Excel

Leyendo las pestanas del Excel que nos interesan agregando el periodo como una nueva columna.

In [76]:
appended_data = []

for i in range(13, 24, 2):
    tab = 'Cuadro '+str(i)
    
    data = pd.read_excel(
        'temp.xls', 
        converters={'column_name': decimal_converter}, 
        sheet_name=tab, 
        skiprows=13, 
        skipfooter=6, 
        header=None, 
        names=columns
    )
    
    # Leyendo el anio
    year_df = pd.read_excel('temp.xls', sheet_name=tab, index_col=None, usecols = "A", header = 5, nrows=0)
    year = year_df.columns.values[0]
    year = year[4:8]
    
    # Agregando el anio al dataframe
    L  = []
    for x in range(data.shape[0]):
        L.append(year)

    data['Año'] = L
    
    # Guardando el DF en la lista
    appended_data.append(data)

# Generando el DF a partir de la lista
df = pd.concat(appended_data)
df.shape

(2381, 81)

In [77]:
print(df['Año'].unique())

['2015' '2016' '2017' '2018' '2019' '2020']


In [87]:
# Moviendo anio a la primera posicion
first_column = df.pop('Año')
  
# Insertando de nuevo
df.insert(0, 'Año', first_column)

df.tail()

,Año,Códigos CPC Vers. 2 A.C.,Concepto,Total oferta a precios comprador,Márgenes de comercio,Márgenes de transporte,Impuestos y derechos a las importaciones,IVA no deducible,Impuestos a los productos (excepto impuestos a importaciones e IVA no deducible),Subvenciones a los productos,...,Actividades de atención de la salud humana y de servicios sociales,"Actividades artísticas, de entretenimiento y recreación y otras actividades de servicios",Actividades de los hogares individuales en calidad de empleadores,Producción a precios básicos Total,Producción a precios básicos Para uso final propio,Producción a precios básicos Otra de no mercado,Producción a precios básicos De mercado,Importaciones Ajustes CIF/FOB sobre importaciones,Importaciones Bienes,Importaciones Servicios
389,2020,960699.0,Otros servicios de diversión y entretenimiento...,4066.0,0.0,0.0,131.0,0.0,0.0,3935.0,...,1738.0,0.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
390,2020,970000.0,Otros servicios personales diversos n.c.p.,11792.0,0.0,1.0,433.0,0.0,0.0,11359.0,...,11158.0,0.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
391,2020,980000.0,Servicios domésticos,4844.0,0.0,0.0,0.0,0.0,0.0,4844.0,...,0.0,4843.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
392,2020,990101.0,Compras directas en el exterior por residentes...,776.0,0.0,0.0,0.0,0.0,0.0,776.0,...,0.0,0.0,NaN,NaN,NaN,776.0,NaN,NaN,NaN,NaN
393,2020,990102.0,Compras directas en el exterior por residentes...,2813.0,0.0,0.0,0.0,0.0,0.0,2813.0,...,0.0,0.0,NaN,NaN,NaN,NaN,2813.0,NaN,NaN,NaN


In [88]:
columns.insert(0, 'Año')

print(columns)

['Año', 'Códigos CPC Vers. 2 A.C.', 'Concepto', 'Total oferta a precios comprador', 'Márgenes de comercio', 'Márgenes de transporte', 'Impuestos y derechos a las importaciones', 'IVA no deducible', 'Impuestos a los productos (excepto impuestos a importaciones e IVA no deducible)', 'Subvenciones a los productos', 'Oferta total precios básicos', 'Agricultura y actividades de servicios conexas ', 'Cultivo permanente de café', 'Ganadería, caza y actividades de servicios conexas ', 'Silvicultura y extracción de madera', 'Pesca y acuicultura', 'Extracción de carbón de piedra y lignito', 'Extracción de petróleo crudo y gas natural y actividades de apoyo para la extracción de petróleo y de gas natural', 'Extracción de minerales metalíferos', 'Extracción de otras minas y canteras', 'Actividades de apoyo para otras actividades de explotación de minas y canteras', 'Procesamiento y conservación de carne y productos cárnicos de bovinos, bufalinos, porcinos y otras carnes n.c.p.; procesamiento y con

## Limpiar y Exportar

Antes de exportar a CSV, se:
- eliminan las columnas innecesarias
- sustituyen los NaN por ceros 
- eliminan las filas que no tienen Codigos CPC

In [79]:
df.drop(columns=['N/A 1', 'N/A 2'], inplace=True)

columns.remove('N/A 1')
columns.remove('N/A 2')

In [80]:
# Cambiando los NaN por ceros
df.fillna(0)

,Año,Códigos CPC Vers. 2 A.C.,Concepto,Total oferta a precios comprador,Márgenes de comercio,Márgenes de transporte,Impuestos y derechos a las importaciones,IVA no deducible,Impuestos a los productos (excepto impuestos a importaciones e IVA no deducible),Subvenciones a los productos,...,Actividades de atención de la salud humana y de servicios sociales,"Actividades artísticas, de entretenimiento y recreación y otras actividades de servicios",Actividades de los hogares individuales en calidad de empleadores,Producción a precios básicos Total,Producción a precios básicos Para uso final propio,Producción a precios básicos Otra de no mercado,Producción a precios básicos De mercado,Importaciones Ajustes CIF/FOB sobre importaciones,Importaciones Bienes,Importaciones Servicios
0,2015,10101.0,Trigo en grano,1217.0,0.0,111.0,0.0,46.0,3.0,0.0,...,0.0,0.0,0.0,7.0,2.0,0.0,5.0,0.0,1050.0,0.0
1,2015,10102.0,Maíz,3832.0,251.0,203.0,131.0,58.0,0.0,0.0,...,0.0,0.0,0.0,940.0,123.0,0.0,817.0,0.0,2249.0,0.0
2,2015,10103.0,Cebada,238.0,21.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,15.0,0.0,0.0,15.0,0.0,197.0,0.0
3,2015,10104.0,Arroz,2406.0,99.0,71.0,0.0,0.0,12.0,0.0,...,0.0,0.0,0.0,2161.0,49.0,0.0,2112.0,0.0,63.0,0.0
4,2015,10199.0,"Sorgo, centeno, avena, mijo y otros cereales n...",66.0,4.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,29.0,0.0,0.0,29.0,0.0,30.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,2020,970000.0,Otros servicios personales diversos n.c.p.,11792.0,0.0,1.0,433.0,0.0,0.0,11359.0,...,11158.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
391,2020,980000.0,Servicios domésticos,4844.0,0.0,0.0,0.0,0.0,0.0,4844.0,...,0.0,4843.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
392,2020,990101.0,Compras directas en el exterior por residentes...,776.0,0.0,0.0,0.0,0.0,0.0,776.0,...,0.0,0.0,0.0,0.0,0.0,776.0,0.0,0.0,0.0,0.0
393,2020,990102.0,Compras directas en el exterior por residentes...,2813.0,0.0,0.0,0.0,0.0,0.0,2813.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2813.0,0.0,0.0,0.0


In [81]:
# Eliminando las filas que no tienen codigo - debido a que no todos los tabs tienen la misma estructura
df.dropna(subset=['Códigos CPC Vers. 2 A.C.'], inplace=True)

In [82]:
df.shape

(2364, 79)

In [10]:
df.to_csv('matriz_oferta_demanda.csv',encoding='latin1', index=False)

## Convirtiendo la matriz en formato de reporte

In [89]:
ou_df = pd.melt(df, id_vars=columns[0:3], value_vars=columns[3:], var_name='Industria', value_name='COP')

In [90]:
# Cambiando los NaN por ceros
ou_df.fillna(0)

,Año,Códigos CPC Vers. 2 A.C.,Concepto,Industria,COP
0,2015,10101.0,Trigo en grano,Total oferta a precios comprador,1217.0
1,2015,10102.0,Maíz,Total oferta a precios comprador,3832.0
2,2015,10103.0,Cebada,Total oferta a precios comprador,238.0
3,2015,10104.0,Arroz,Total oferta a precios comprador,2406.0
4,2015,10199.0,"Sorgo, centeno, avena, mijo y otros cereales n...",Total oferta a precios comprador,66.0
...,...,...,...,...,...
179659,2020,960699.0,Otros servicios de diversión y entretenimiento...,Importaciones Servicios,0.0
179660,2020,970000.0,Otros servicios personales diversos n.c.p.,Importaciones Servicios,0.0
179661,2020,980000.0,Servicios domésticos,Importaciones Servicios,0.0
179662,2020,990101.0,Compras directas en el exterior por residentes...,Importaciones Servicios,0.0


In [91]:
ou_df.shape

(179664, 5)

In [92]:
ou_df.to_csv('reporte_oferta_demanda.csv',encoding='latin1', index=False)